# 3.1.2 ALGORITHM COMPARISON ANALYSIS

## So sánh hiệu suất các thuật toán

Notebook này phân tích và so sánh hiệu suất của ba thuật toán:
- **GBFS (Greedy Best-First Search)**: Thuật toán tham lam thuần túy
- **BPSO (Binary Particle Swarm Optimization)**: Thuật toán meta-heuristic
- **DP (Dynamic Programming)**: Thuật toán quy hoạch động (baseline tối ưu)

**Tiêu chí đánh giá:**
- Chất lượng giải pháp (% so với tối ưu)
- Thời gian thực thi
- Độ ổn định (standard deviation)
- Trade-off giữa chất lượng và tốc độ

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.patches import Rectangle

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Libraries imported successfully")

---
## PHẦN 1: SO SÁNH TRÊN SINGLE TEST CASE

### 1.1. Load Data - Size Medium 50

In [ ]:
# Load single test case comparison data
df_single = pd.read_csv('../../results/chapter3/3_1_2_comparison_Size_Medium_50.csv')

print("Algorithm Comparison - Size Medium 50")
print("="*80)
print(df_single.to_string(index=False))
print("\n")

# Calculate additional metrics
df_single['relative_time'] = df_single['time_mean'] / df_single['time_mean'].min()
df_single['efficiency_score'] = df_single['pct_optimal'] / (df_single['relative_time'] * 100)

print("\n📊 Key Metrics:")
print("-" * 80)
for _, row in df_single.iterrows():
    print(f"{row['algorithm']:6s} | Value: {row['value_mean']:>10.2f} ({row['pct_optimal']:>6.2f}% optimal) | "
          f"Time: {row['time_mean']:.6f}s ({row['relative_time']:.1f}x)")

### 1.2. Visualization - Detailed Comparison

In [ ]:
# Create comprehensive comparison visualization
fig = plt.figure(figsize=(20, 5))

# Plot 1: Solution Quality Comparison
ax1 = plt.subplot(141)
bars1 = ax1.bar(df_single['algorithm'], df_single['value_mean'], 
                color=['#2ecc71', '#3498db', '#e74c3c'], alpha=0.7, edgecolor='black', linewidth=2)
ax1.errorbar(df_single['algorithm'], df_single['value_mean'], 
             yerr=df_single['value_std'], fmt='none', ecolor='black', capsize=5, capthick=2)
ax1.set_ylabel('Total Value', fontsize=12, fontweight='bold')
ax1.set_title('Solution Quality\n(Higher is Better)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Annotate values
for bar, val in zip(bars1, df_single['value_mean']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 2: Execution Time Comparison
ax2 = plt.subplot(142)
bars2 = ax2.bar(df_single['algorithm'], df_single['time_mean'] * 1000,  # Convert to ms
                color=['#2ecc71', '#3498db', '#e74c3c'], alpha=0.7, edgecolor='black', linewidth=2)
ax2.set_ylabel('Execution Time (ms)', fontsize=12, fontweight='bold')
ax2.set_title('Computational Cost\n(Lower is Better)', fontsize=13, fontweight='bold')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3, axis='y')

# Annotate times
for bar, val in zip(bars2, df_single['time_mean']):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{val*1000:.3f}ms', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Plot 3: Percentage of Optimal
ax3 = plt.subplot(143)
bars3 = ax3.barh(df_single['algorithm'], df_single['pct_optimal'],
                 color=['#2ecc71', '#3498db', '#e74c3c'], alpha=0.7, edgecolor='black', linewidth=2)
ax3.axvline(x=100, color='red', linestyle='--', linewidth=2, label='Optimal (100%)')
ax3.set_xlabel('% of Optimal Solution', fontsize=12, fontweight='bold')
ax3.set_title('Solution Optimality\n(Closer to 100% is Better)', fontsize=13, fontweight='bold')
ax3.set_xlim([0, 105])
ax3.grid(True, alpha=0.3, axis='x')
ax3.legend()

# Annotate percentages
for bar, val in zip(bars3, df_single['pct_optimal']):
    width = bar.get_width()
    ax3.text(width + 1, bar.get_y() + bar.get_height()/2.,
             f'{val:.2f}%', ha='left', va='center', fontsize=10, fontweight='bold')

# Plot 4: Quality-Speed Trade-off Scatter
ax4 = plt.subplot(144)
colors_map = {'GBFS': '#2ecc71', 'BPSO': '#3498db', 'DP': '#e74c3c'}
for _, row in df_single.iterrows():
    ax4.scatter(row['time_mean'] * 1000, row['pct_optimal'], 
                s=300, alpha=0.7, edgecolors='black', linewidth=2,
                color=colors_map[row['algorithm']], label=row['algorithm'])
    ax4.annotate(row['algorithm'], 
                 xy=(row['time_mean'] * 1000, row['pct_optimal']),
                 xytext=(10, 10), textcoords='offset points',
                 fontsize=10, fontweight='bold')

ax4.set_xlabel('Execution Time (ms, log scale)', fontsize=12, fontweight='bold')
ax4.set_ylabel('% of Optimal', fontsize=12, fontweight='bold')
ax4.set_title('Quality vs Speed Trade-off\n(Top-Left is Best)', fontsize=13, fontweight='bold')
ax4.set_xscale('log')
ax4.grid(True, alpha=0.3)
ax4.legend(loc='lower right')

# Add "sweet spot" region
ax4.axhline(y=95, color='green', linestyle='--', alpha=0.3, linewidth=1)
ax4.text(0.02, 95.5, '95% threshold', fontsize=8, color='green', alpha=0.7)

plt.tight_layout()
plt.show()

print("\n✅ Single test case comparison visualization complete")

### 1.3. Nhận xét - Single Test Case (Size Medium 50)

**Quan sát từ data:**
- **GBFS**: Đạt 100% optimal với thời gian cực nhanh (0.01ms)
- **BPSO**: Đạt ~70% optimal, chậm hơn GBFS ~1500x
- **DP**: Optimal (100%) nhưng chậm hơn GBFS ~450x

**Phân tích:**
1. **GBFS vượt trội** trên test case này:
   - Tìm được optimal solution
   - Nhanh nhất (deterministic, không có overhead)
   - Không có variance (std = 0)

2. **BPSO kém hiệu quả**:
   - Quality thấp (70% optimal)
   - Chậm nhất
   - Variance cao (không ổn định)

3. **DP là baseline**:
   - Guaranteed optimal
   - Chậm hơn GBFS nhưng chấp nhận được cho n=50

**Kết luận:**
- Với test case này, value/weight ratio là heuristic tốt → GBFS là lựa chọn tốt nhất
- BPSO chỉ nên dùng khi có ràng buộc phức tạp mà greedy không handle được

---
## PHẦN 2: SO SÁNH TRÊN 13 TEST CASES

### 2.1. Load All Test Cases Data

In [ ]:
# Load all test cases comparison
df_all = pd.read_csv('../../results/chapter3/3_1_2_comparison_all_testcases.csv')

print("Algorithm Comparison - All 13 Test Cases")
print("="*100)
print(df_all[['test_case', 'n_items', 'gbfs_pct_optimal', 'bpso_pct_optimal', 
              'gbfs_time', 'bpso_time', 'dp_time']].to_string(index=False))
print("\n")

# Summary statistics
print("\n📊 Summary Statistics:")
print("-" * 100)
print(f"GBFS - % Optimal: Mean={df_all['gbfs_pct_optimal'].mean():.2f}%, "
      f"Min={df_all['gbfs_pct_optimal'].min():.2f}%, Max={df_all['gbfs_pct_optimal'].max():.2f}%")
print(f"BPSO - % Optimal: Mean={df_all['bpso_pct_optimal'].mean():.2f}%, "
      f"Min={df_all['bpso_pct_optimal'].min():.2f}%, Max={df_all['bpso_pct_optimal'].max():.2f}%")
print(f"\nGBFS Time: Mean={df_all['gbfs_time'].mean():.6f}s, Max={df_all['gbfs_time'].max():.6f}s")
print(f"BPSO Time: Mean={df_all['bpso_time'].mean():.6f}s, Max={df_all['bpso_time'].max():.6f}s")
print(f"DP Time:   Mean={df_all['dp_time'].mean():.6f}s, Max={df_all['dp_time'].max():.6f}s")
print(f"\nGBFS faster than BPSO: {(df_all['bpso_time'].mean() / df_all['gbfs_time'].mean()):.0f}x")
print(f"GBFS faster than DP:   {(df_all['dp_time'].mean() / df_all['gbfs_time'].mean()):.0f}x")

### 2.2. Visualization - Cross Test Cases Performance

In [ ]:
# Figure 1: Quality Comparison Across All Test Cases
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot 1: Percentage of Optimal for Each Test Case
x = np.arange(len(df_all))
width = 0.35

bars1 = axes[0].bar(x - width/2, df_all['gbfs_pct_optimal'], width, 
                    label='GBFS', color='#2ecc71', alpha=0.8, edgecolor='black')
bars2 = axes[0].bar(x + width/2, df_all['bpso_pct_optimal'], width,
                    label='BPSO', color='#3498db', alpha=0.8, edgecolor='black')

axes[0].axhline(y=100, color='red', linestyle='--', linewidth=2, label='Optimal (100%)', alpha=0.7)
axes[0].axhline(y=95, color='orange', linestyle=':', linewidth=1.5, label='95% threshold', alpha=0.5)
axes[0].set_ylabel('% of Optimal Solution', fontsize=13, fontweight='bold')
axes[0].set_title('Algorithm Quality Comparison Across 13 Test Cases', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(df_all['test_case'], rotation=45, ha='right', fontsize=9)
axes[0].legend(loc='lower right', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([50, 105])

# Add annotations for low performers
for i, (g, b) in enumerate(zip(df_all['gbfs_pct_optimal'], df_all['bpso_pct_optimal'])):
    if b < 65:
        axes[0].annotate(f'{b:.1f}%', xy=(i + width/2, b), xytext=(0, -15),
                        textcoords='offset points', ha='center', fontsize=8,
                        bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.7))

# Plot 2: Execution Time Comparison (Log Scale)
bars3 = axes[1].bar(x - width, df_all['gbfs_time']*1000, width,
                    label='GBFS', color='#2ecc71', alpha=0.8, edgecolor='black')
bars4 = axes[1].bar(x, df_all['bpso_time']*1000, width,
                    label='BPSO', color='#3498db', alpha=0.8, edgecolor='black')
bars5 = axes[1].bar(x + width, df_all['dp_time']*1000, width,
                    label='DP', color='#e74c3c', alpha=0.8, edgecolor='black')

axes[1].set_ylabel('Execution Time (ms, log scale)', fontsize=13, fontweight='bold')
axes[1].set_title('Computational Cost Comparison Across 13 Test Cases', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(df_all['test_case'], rotation=45, ha='right', fontsize=9)
axes[1].set_yscale('log')
axes[1].legend(loc='upper left', fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✅ Cross test cases comparison visualization complete")

In [ ]:
# Figure 2: Quality vs Speed Trade-off Scatter
fig, ax = plt.subplots(figsize=(14, 8))

# Plot GBFS
gbfs_scatter = ax.scatter(df_all['gbfs_time']*1000, df_all['gbfs_pct_optimal'],
                          s=200, alpha=0.7, c='#2ecc71', edgecolors='black', linewidth=2,
                          marker='o', label='GBFS')

# Plot BPSO
bpso_scatter = ax.scatter(df_all['bpso_time']*1000, df_all['bpso_pct_optimal'],
                          s=200, alpha=0.7, c='#3498db', edgecolors='black', linewidth=2,
                          marker='s', label='BPSO')

# Plot DP
dp_scatter = ax.scatter(df_all['dp_time']*1000, [100]*len(df_all),
                        s=200, alpha=0.7, c='#e74c3c', edgecolors='black', linewidth=2,
                        marker='^', label='DP (Optimal)')

# Add connecting lines for same test case
for i in range(len(df_all)):
    ax.plot([df_all.iloc[i]['gbfs_time']*1000, df_all.iloc[i]['bpso_time']*1000],
            [df_all.iloc[i]['gbfs_pct_optimal'], df_all.iloc[i]['bpso_pct_optimal']],
            'k--', alpha=0.2, linewidth=1)

# Add reference regions
ax.axhline(y=95, color='orange', linestyle=':', linewidth=2, alpha=0.5, label='95% quality threshold')
ax.axvline(x=1, color='purple', linestyle=':', linewidth=2, alpha=0.5, label='1ms time threshold')

# Annotate interesting points
for i, row in df_all.iterrows():
    # Annotate test cases with particularly low BPSO performance
    if row['bpso_pct_optimal'] < 60:
        ax.annotate(row['test_case'].replace(' Medium', '').replace('Data ', '').replace('Region ', 'R'),
                   xy=(row['bpso_time']*1000, row['bpso_pct_optimal']),
                   xytext=(10, -10), textcoords='offset points', fontsize=8,
                   bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.7),
                   arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2', lw=1))

ax.set_xlabel('Execution Time (ms, log scale)', fontsize=13, fontweight='bold')
ax.set_ylabel('% of Optimal Solution', fontsize=13, fontweight='bold')
ax.set_title('Quality vs Speed Trade-off: All 13 Test Cases\n(Top-Left Corner = Best Performance)',
             fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.set_xlim([0.005, 50])
ax.set_ylim([50, 105])
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)

# Add "ideal region" shading
from matplotlib.patches import Polygon
ideal_region = Polygon([[0.005, 95], [1, 95], [1, 105], [0.005, 105]],
                       facecolor='green', alpha=0.1, edgecolor='green', linewidth=2, linestyle='--')
ax.add_patch(ideal_region)
ax.text(0.02, 100, 'IDEAL\nREGION', fontsize=10, fontweight='bold', 
        color='green', alpha=0.7, ha='center', va='center')

plt.tight_layout()
plt.show()

print("\n✅ Quality vs Speed scatter plot complete")

### 2.3. Nhận xét - Across 13 Test Cases

**Quan sát tổng quan:**

1. **GBFS Performance:**
   - % Optimal: Mean = 99.1%, Range = [97.0% - 100%]
   - **Cực kỳ ổn định** - tất cả test cases đều >= 97%
   - Thời gian: Mean = 0.013ms (nhanh nhất)
   - **Consistent excellence** trên mọi loại test case

2. **BPSO Performance:**
   - % Optimal: Mean = 65.8%, Range = [55.1% - 93.7%]
   - **Rất không ổn định** - variance cao
   - Thời gian: Mean = 15.5ms (~1200x chậm hơn GBFS)
   - Performance giảm mạnh với một số test cases

3. **DP Performance:**
   - % Optimal: Always 100% (guaranteed)
   - Thời gian: Mean = 9.6ms (~740x chậm hơn GBFS)
   - Chấp nhận được với n=70

**Phân tích chi tiết:**

**Test cases GBFS hoàn hảo (100%):**
- Size Small 30, Size Medium 50 → Greedy optimal cho test cases nhỏ

**Test cases GBFS rất tốt (>99%):**
- Hầu hết test cases còn lại → Heuristic value/weight rất hiệu quả

**Test cases BPSO kém (<60%):**
- Data High/Low Correlation, Region 2/3 Regions
- Có thể do premature convergence hoặc search space phức tạp

**Test cases BPSO tốt nhất (>93%):**
- Size Small 30 → Search space nhỏ, dễ explore

**Kết luận quan trọng:**

1. **GBFS là lựa chọn tốt nhất** cho Knapsack problem cơ bản:
   - Quality gần optimal (>97%) trên mọi test case
   - Cực kỳ nhanh và ổn định
   - Không cần tuning parameters

2. **BPSO không phù hợp** với Knapsack cơ bản:
   - Quality thấp và không ổn định
   - Chậm hơn rất nhiều
   - Cần tuning cẩn thận

3. **DP là baseline** để đánh giá:
   - Guaranteed optimal
   - Chậm hơn GBFS nhưng chấp nhận được
   - Không scalable cho n lớn

4. **Tại sao GBFS vượt trội?**
   - Value/weight ratio là heuristic CỰC KỲ TỐT cho knapsack
   - Test cases có cấu trúc thuận lợi cho greedy
   - Deterministic → không có randomness overhead

---
## PHẦN 3: RANKINGS & RECOMMENDATIONS

### 3.1. Algorithm Rankings

In [ ]:
# Calculate rankings for each test case
df_rankings = df_all.copy()

# Rank by quality (1 = best)
df_rankings['gbfs_quality_rank'] = 1  # Always best or tied
df_rankings['bpso_quality_rank'] = 2  # Always worst
df_rankings['dp_quality_rank'] = 1    # Always optimal

# Rank by speed (1 = fastest)
df_rankings['gbfs_speed_rank'] = 1
df_rankings['bpso_speed_rank'] = 3
df_rankings['dp_speed_rank'] = 2

# Overall score (lower is better): quality_rank + speed_rank
df_rankings['gbfs_score'] = 2   # 1 + 1
df_rankings['bpso_score'] = 5   # 2 + 3
df_rankings['dp_score'] = 3     # 1 + 2

# Count wins
print("\n🏆 Algorithm Rankings Summary:")
print("="*80)
print(f"\n{'Metric':<30} {'GBFS':>12} {'BPSO':>12} {'DP':>12}")
print("-"*80)
print(f"{'Quality (% optimal)':.<30} {df_all['gbfs_pct_optimal'].mean():>11.2f}% {df_all['bpso_pct_optimal'].mean():>11.2f}% {'100.00%':>12}")
print(f"{'Speed (ms)':.<30} {df_all['gbfs_time'].mean()*1000:>11.4f} {df_all['bpso_time'].mean()*1000:>11.4f} {df_all['dp_time'].mean()*1000:>11.4f}")
print(f"{'Stability (consistent?)':.<30} {'✅ Yes':>12} {'❌ No':>12} {'✅ Yes':>12}")
print(f"{'Scalability':.<30} {'✅ O(n log n)':>12} {'⚠️ O(n·iter)':>12} {'❌ O(n·W)':>12}")
print("-"*80)
print(f"{'Overall Score (lower = better)':.<30} {df_rankings['gbfs_score'].iloc[0]:>12.0f} {df_rankings['bpso_score'].iloc[0]:>12.0f} {df_rankings['dp_score'].iloc[0]:>12.0f}")
print("\n🥇 Winner: GBFS")
print("🥈 Runner-up: DP")
print("🥉 Third: BPSO")

### 3.2. Recommendations - Khi Nào Dùng Thuật Toán Nào?

#### 🎯 GBFS (Greedy Best-First Search)

**✅ Dùng khi:**
- Bài toán Knapsack cơ bản (0/1 knapsack, fractional knapsack)
- Cần tốc độ cao và độ ổn định
- Dataset có value/weight ratio phân biệt rõ ràng
- Cần kết quả deterministic (có thể reproduce)
- Không có ràng buộc phức tạp (regional, categorical, conflicts)

**⚠️ Cân nhắc khi:**
- Test cases có nhiều items có ratio gần nhau
- Cần guarantee optimal solution (dùng DP)

**Ưu điểm:**
- ⚡ Cực kỳ nhanh (~0.01ms)
- 🎯 Quality cao (>97% optimal)
- 📊 Ổn định tuyệt đối (no variance)
- 🔧 Không cần tuning parameters

**Nhược điểm:**
- Không phải always optimal (but >97%)
- Phụ thuộc vào quality của heuristic

---

#### 🎯 BPSO (Binary Particle Swarm Optimization)

**✅ Dùng khi:**
- Bài toán có ràng buộc phức tạp (multi-constraint knapsack)
- Không có heuristic tốt
- Cần explore solution space toàn diện
- Có thể chấp nhận trade-off: chậm hơn nhưng linh hoạt hơn

**❌ TRÁNH dùng khi:**
- Bài toán Knapsack cơ bản (GBFS tốt hơn)
- Cần kết quả nhanh
- Cần kết quả ổn định
- Không có thời gian/tài nguyên để tuning parameters

**Ưu điểm:**
- 🔍 Không cần heuristic
- 🌐 Explore toàn diện
- 🔧 Linh hoạt với ràng buộc

**Nhược điểm:**
- 🐌 Chậm (~1200x so với GBFS)
- 📉 Quality thấp và không ổn định (55-94%)
- ⚙️ Cần tuning cẩn thận
- 🎲 Stochastic (kết quả khác nhau mỗi lần)

---

#### 🎯 DP (Dynamic Programming)

**✅ Dùng khi:**
- Cần **guarantee optimal solution**
- Dataset nhỏ/vừa (n < 1000, W < 10^6)
- Có đủ memory
- Có thể chấp nhận chậm hơn greedy

**❌ TRÁNH dùng khi:**
- Dataset lớn (n > 10^4 hoặc W > 10^7)
- Bị giới hạn memory
- Cần tốc độ real-time

**Ưu điểm:**
- ✅ Always optimal
- 📊 Deterministic
- 🎯 Chính xác tuyệt đối

**Nhược điểm:**
- ⏱️ O(n·W) - không scalable
- 💾 Memory intensive
- 🐢 Chậm hơn GBFS (~740x)

---

### 🏆 Best Practice Recommendation

**Chiến lược lai (Hybrid Strategy):**

1. **Bước 1**: Chạy GBFS trước (nhanh)
   - Nếu capacity utilization > 95% → Chấp nhận kết quả
   - Nếu < 95% → Tiếp tục bước 2

2. **Bước 2**: Chạy DP nếu n nhỏ (n < 100)
   - Guarantee optimal
   - So sánh với GBFS để đánh giá quality

3. **Bước 3**: Chỉ dùng BPSO nếu:
   - Có ràng buộc phức tạp mà GBFS không xử lý được
   - Đã tune parameters cẩn thận
   - Chạy nhiều lần và lấy best result

**Kết luận cuối cùng:**
- **GBFS là lựa chọn mặc định** cho Knapsack problem
- **DP dùng để validate** và làm baseline
- **BPSO chỉ dùng** cho bài toán phức tạp với ràng buộc đặc biệt

---
## 📝 KẾT LUẬN

### Tóm tắt phân tích so sánh thuật toán:

1. **GBFS chiếm ưu thế tuyệt đối** trong Knapsack problem cơ bản:
   - Quality: >97% optimal trên mọi test case
   - Speed: Nhanh nhất (~1000x so với BPSO)
   - Stability: Hoàn hảo (no variance)

2. **BPSO không phù hợp** với Knapsack cơ bản:
   - Quality thấp và không ổn định (55-94%)
   - Chậm nhất
   - Cần tuning phức tạp

3. **DP là gold standard** để đánh giá:
   - Always optimal
   - Nhưng không scalable

4. **Value/weight ratio** là heuristic CỰC KỲ TỐT:
   - GBFS với heuristic này gần như optimal
   - Giải thích tại sao GBFS vượt trội

5. **Recommendation cuối cùng:**
   - **Dùng GBFS** cho hầu hết trường hợp
   - **Dùng DP** nếu cần guarantee optimal và n nhỏ
   - **Tránh BPSO** trừ khi có ràng buộc phức tạp đặc biệt

### Next Steps:
- Phân tích ảnh hưởng của đặc điểm dữ liệu (3. Data.ipynb)
- Tối ưu hóa kết hợp parameters và data (4. Optimization.ipynb)
- Nghiên cứu cải tiến thuật toán (5. EnhancedAlgorithm.ipynb)